## Import Libraries

In [ ]:
import os
import numpy as np 
from astropy.io import fits 
from smooth_kevin import smoother
import py_specrebin_vec
import matplotlib.pyplot as plt 
from matplotlib import rc
import py_specrebin
import pandas as pd
path_name = '.'

<hr/>

**INSTRUCTION**: <br />
NOTE:Notebook is separated into three sections. They are separated by the three #BREAK: <br /> 
    -AGST Subtraction <br />
    -BPT Ratios <br />
    -Creating BPT Diagram

-**AGST (AirGlow and StarLight Contamincation) Subtraction** <br/>
1.Create a folder called "AGST Subtracted Spectra" (Don't include the quotation mark) in the same space in which you're running this code. Results will be save here.<br />
2.Change mask_name to the appropriate mask you're planning on analyzing.<br />
3.Below the "Saving ST and AG subtracted spectra" section is the first #BREAK. Run everything from there and above.<br />
4.Give it a few minutes. The rebinning function will take around 4-5 minutes to run.<br /> 
5.Resulting airglow and starlight subtracted spectra will be saved as a 2d array formated as such:
MaskName_AGST_Subtracted_Spectra.fits.gz <br />
6.If you're planning on just getting the AGST spectra, you can continue with a new mask by changing mask_name. If you're planning on getting BPT Ratios continue to next section.

-**BPT Ratios** <br/>
NOTE: This section only work for M33 mask with 600ZD grating. 1200G doesn't include H-beta and OIII emission lines.<br/>
1.Create a folder called "BPT Diagram CSV" <br/>
2.Below "Add Redshift to CSV" section is the second #BREAK. Run everything from there to the first #BREAK. <br/>
3.This will find the rebinned flux of H-Alpha, NII, H-Beta, and OIII -> Take the ratios -> Remove any of the NaN and inf ratios, and save the remaining float (decimal numbers) ratios -> Find the redshift and QOP of the remaining ratios using index -> Check to see if a CSV file has been created to save these. If not, create a CSV file and save the mask name, ratios, redshift, and QOP. If yes, open the existing CSV file and add to it. <br/> 
4.Do this for all mask before proceeding to the third section.

-**Creating BPT Diagram** <br/>
NOTE: Do not run this section unless you have run section 1 and 2 for all the masks you wish to include. This section will utilize the CSV file you created in section 2.<br/>
1.Run just the "Create BPT Diagram" section.<br/>
2.This will produce and save two diagrams. Each titled "BPT Diagram" and "BPT_Diagram (QOP>=2)". These are BPT diagram, one with QOP of 3 and one with QOP of 2 and 3, respectively.<br/> 

<hr/>

**Issues**: <br/>
Mask D1M33P: ratio_pairs,index_of_pairs have different length. Can't add to CSV. Fix by changing... <br/>
index_of_pairs = np.arange(len(slit_nums))[np.isin(pairing,np.array(ratio_pairs))[:,1]] <br/>
from 0 to 1

pTN1bspec1d slit #171 causing issues. <br/>
VerifyError: Unparsable card (CD1_1), fix it first with .verify('fix') <br/>
Removed slit from process. Not sure how to fix error. <br/>
With slit removed. New error arise when rebinning.<br/>
IndexError: index 0 is out of bounds for axis 0 with size 0<br/>
Skip mask. 
<hr/>

## Variable Pannel

In [ ]:
mask_name = "pTS3spec1d"
grating = 1200

## Define Wavelength

In [ ]:
new_wave_600 = np.arange(4000, 11000, .65) 
new_wave_1200 = np.arange(6000, 11000, .33) 

## Function To Get Data From FITS File

In [ ]:
# Saving the original streams for stdout and stderr. To be used for logging output later
import sys
std_out = sys.stdout; std_err = sys.stderr

In [ ]:
def get_original_data(file_names,mask_name):
    
    tot_flux = []
    tot_wave = []
    tot_ivar = []
    
    for j in range(len(file_names)):
        #read in star data
        h_star = fits.open(path_name + '/' + 'data/{0}'.format(mask_name) + '/' + file_names[j], ignore_missing_end = True)
        
        data_star1 = h_star[1].data
        star_flux1 = data_star1['SKYSPEC'][0]
        star_wave1 = data_star1['LAMBDA'][0]
        star_ivar1 = data_star1['IVAR'][0]
        
        data_star2 = h_star[2].data
        star_flux2 = data_star2['SKYSPEC'][0]
        star_wave2 = data_star2['LAMBDA'][0]
        star_ivar2 = data_star2['IVAR'][0]
        
        
        #combine the blue and red side into one list
        star_flux = np.array(list(star_flux1) + list(star_flux2))
        star_wave = np.array(list(star_wave1) + list(star_wave2))
        star_ivar = np.array(list(star_ivar1) + list(star_ivar2))
        
        if (sum(star_flux) == 0 and sum(star_ivar) == 0 and sum(star_wave) == 0): #section added so that any slit w/o data will use serendip data instead
            try: #some slit do not have data BUT also have no serendip. In that case, just add empty data
                file_name_split = file_names[j].split(".")
                serendip_file_name = "{0}.{1}.{2}.serendip1.{3}.{4}".format(file_name_split[0],file_name_split[1],
                                                                       file_name_split[2],file_name_split[4],file_name_split[5])
                path_to_serendip = fits.open(path_name + '/' + "data/{0}/{1}".format(mask_name,serendip_file_name))

                star_flux1_serendip = path_to_serendip[1].data["SKYSPEC"][0]
                star_flux2_serendip = path_to_serendip[2].data["SKYSPEC"][0]
                star_flux_serendip = np.concatenate((star_flux1_serendip,star_flux2_serendip))

                star_ivar1_serendip = path_to_serendip[1].data["IVAR"][0]
                star_ivar2_serendip = path_to_serendip[2].data["IVAR"][0]
                star_ivar_serendip = np.concatenate((star_ivar1_serendip,star_ivar2_serendip))

                star_wave1_serendip = path_to_serendip[1].data["LAMBDA"][0]
                star_wave2_serendip = path_to_serendip[2].data["LAMBDA"][0]
                star_wave_serendip = np.concatenate((star_wave1_serendip,star_wave2_serendip))

                tot_flux.append(star_flux_serendip)
                tot_wave.append(star_wave_serendip)
                tot_ivar.append(star_ivar_serendip)

                h_star.close()
                
            except: #For error about serendip file not existing
                #add to above lists
                tot_flux.append(star_flux)
                tot_wave.append(star_wave)
                tot_ivar.append(star_ivar)

                h_star.close()
        
        else:
            #add to above lists
            tot_flux.append(star_flux)
            tot_wave.append(star_wave)
            tot_ivar.append(star_ivar)

            h_star.close()
        
    return tot_flux, tot_wave, tot_ivar 

## Function For Rebinning w/ Updated Rebinning Process

In [ ]:
def rebin(fluxes, waves, ivar, grating):
    
    rbflux = []
    rbivar = []
    
    if grating == 600:
        new_wave = new_wave_600
    elif grating == 1200:
        new_wave = new_wave_1200
    
    for i in range(len(waves)):
        new_flux,new_ivar = py_specrebin_vec.rebinspec(waves[i],fluxes[i],new_wave,ivar=ivar[i])
        new_flux_err = 1/np.sqrt(new_ivar)

        rbflux.append(new_flux)
        rbivar.append(new_ivar)
        
    return rbflux, new_wave, rbivar

## Function To Create Median

In [ ]:
def find_median(rebinned_flux_array):
    
    median_vals = []
    
    for i in range(len(rebinned_flux_array[0])):

        comp = []
        
        for array in rebinned_flux_array:
            
            if np.isfinite(array[i]) == True:
                comp.append(array[i])
                
        median_vals.append(np.median(comp))
        
    return median_vals

## Function To Read ISM_EM_LINES.txt & Extract Slit # of Excluded Slits

In [ ]:
def get_exclusions():
    filepath = 'ISM_EM_LINES.txt'
    fp = open(filepath)
    all_data = []
    for line in (fp):
        mask_name = line.split(':')[0].split('_')[0]
        slit_number = line.split(':')[1].strip().split(" ")[0]
        if len(slit_number) == 2:
            slit_number = '0' + slit_number
        elif len(slit_number) == 1:
            slit_number = '00' + slit_number
        else:
            pass
        object_id = line.split(':')[1].strip().split()[1]
        data = {}
        data['mask_name'] = mask_name
        data['slit_number'] = slit_number
        data['object_id'] = object_id
        all_data.append(data)
    return all_data     

In [ ]:
def get_files_to_include(folder):
    import os
    list_of_files_to_include = []
    list_of_files_to_exclude = []
    serendip_files = []
    all_file_names_in_folder = os.listdir('data/{}'.format(folder))
    y = len(all_file_names_in_folder)
    print("The number of files in the folder is {0}".format(y))
    all_data = get_exclusions()
    len_all_data = len(all_data)
    for n in range(y):
        parts_of_file_name = all_file_names_in_folder[n].split(".")
        if parts_of_file_name[0] == 'spec1d': # avoids hidden DS_Store files on my mac
            object_id = parts_of_file_name[3]
            slit_number = parts_of_file_name[2]
            mask_name = parts_of_file_name[1]
            should_include = True
            should_exclude = True
            for k in range(len_all_data):
                if ((object_id == all_data[k]['object_id']) and (slit_number == all_data[k]['slit_number']) and (mask_name == all_data[k]['mask_name'])):
                    should_include = False
                    should_exclude = True
                if 'serendip' in object_id:
                    should_include = False
                    should_exclude = False
            if should_include == True:
                list_of_files_to_include.append(all_file_names_in_folder[n])       
            elif should_exclude == True:
                list_of_files_to_exclude.append(all_file_names_in_folder[n])
            elif should_include == False & should_exclude == False:
                serendip_files.append(all_file_names_in_folder[n])
    
    print('The number of files left after exclusions is {0}'.format(len(list_of_files_to_include)))
    
    return sorted(list_of_files_to_include), sorted(list_of_files_to_exclude), sorted(serendip_files)


In [ ]:
def get_slit_nums(files):
    
    slit_nums = []
    
    if len(files) > 1:
    
        for i in range(len(files)):
            parts_of_file_name = files[i].split(".")
            slit_num = parts_of_file_name[2]
            slit_nums.append(int(slit_num))
            
    return slit_nums

## Calls To Get Slit Numbers

In [ ]:
#filtering files
list_of_files_to_include, list_of_files_to_exclude, list_of_serendip_files = get_files_to_include(mask_name)

file_names = list_of_files_to_include
file_names_exclude = list_of_files_to_exclude
file_names_serendip = list_of_serendip_files
file_names_all = list_of_files_to_include + list_of_files_to_exclude

In [ ]:
slit_nums = get_slit_nums(file_names) #get slit # of INCLUDED slits
slit_nums_exclude = get_slit_nums(file_names_exclude) #get slit # of EXCLUDED slits
all_slit_nums = get_slit_nums(file_names_all) #slit # of INCLUDED & EXCLUDED slits

print("Slit # to INCLUDE in median calculation: {0}".format(slit_nums))
print("Slit # to EXCLUDE: {0}".format(slit_nums_exclude))

## Calls For Data Extraction & Rebinning

In [ ]:
#getting data
#try getting and rebinning all files
flux, wave, ivar = get_original_data(file_names, mask_name) 

In [ ]:
#getting all excluded data
flux_exclude, wave_exclude, ivar_exclude = get_original_data(file_names_exclude, mask_name)

In [ ]:
#rebinning the original data
rbflux, rbwave, rbivar = rebin(flux, wave, ivar, grating) # this takes about 4 minutes to run

In [ ]:
#rebinning the excluded data
rbflux_exclude, rbwave_exclude, rbivar_exclude = rebin(flux_exclude, wave_exclude, ivar_exclude, grating)

## Call For Median Creation

In [ ]:
median = find_median(rbflux) #taking the median

## Moving Median

In [ ]:
from scipy.ndimage import median_filter

def moving_median(a, size=325):
    
    '''
    Returns the moving median values of the array,
    using a window of a given size, centered at
    each point.
    
    Version - 4.0
    
    Parameters
    ----------
    a : ndarray
        One dimensional flux array.
    window : int, optional
        The size of each segment for taking the median.
        
    Returns
    ----------
    median_arr : One dimensional array of moving median.
    
    '''
        
    all_indices = np.arange(len(a))
    finite_bool = np.isfinite(a)
    nan_indices = all_indices[np.invert(finite_bool)]
    nan_indices_set = set(nan_indices)
    n = len(finite_bool)

    if (nan_indices_set=={0,n} or nan_indices_set=={0} or nan_indices_set=={n}):
        
        finite_indices = all_indices[finite_bool]
        nearest_finite_indices = np.searchsorted(finite_indices, nan_indices)
        nearest_finite_indices = nearest_finite_indices - (nearest_finite_indices==len(finite_indices))
        a[nan_indices] = a[finite_indices[nearest_finite_indices]][:]
        median_arr = median_filter(a, size, mode='nearest')

    elif (len(nan_indices_set)==0):
        
        median_arr = np.nan*np.ones(len(a))

    else:
        
        if True not in finite_bool:
            median_arr = np.nan*np.ones(len(a))
            
        else:
            finite_indices = all_indices[finite_bool]
            nearest_finite_indices = np.searchsorted(finite_indices, nan_indices)
            gap_indices = ((nearest_finite_indices>0) & (nearest_finite_indices<len(finite_indices)))
            middle_nan_indices = nan_indices[gap_indices]
            right_nearest_indices = finite_indices[nearest_finite_indices[gap_indices]]
            left_nearest_indices = finite_indices[nearest_finite_indices[gap_indices] - 1]
            right_distances = abs(right_nearest_indices - middle_nan_indices)
            left_distances = abs(left_nearest_indices - middle_nan_indices)
            right_is_near_bool = (left_distances > right_distances)
            left_is_near_bool = (left_distances <= right_distances)
            a[middle_nan_indices[right_is_near_bool]] = a[right_nearest_indices[right_is_near_bool]][:]
            a[middle_nan_indices[left_is_near_bool]] = a[left_nearest_indices[left_is_near_bool]][:]
            a[nan_indices[nearest_finite_indices==0]] = a[finite_indices[0]]
            a[nan_indices[nearest_finite_indices==len(finite_indices)]] = a[finite_indices[-1]]
            median_arr = median_filter(a, size, mode='nearest')
    
    return (median_arr)

## Removing Airglow and Starlight Contamination

In [ ]:
def AGST_subtraction(mask_name,slit_nums,rebinned_flux,median): #airglow and starlight subtraction
    
    '''
    
    Parameters
    ----------
    mask_name : str, required
        Name of mask.
    slit_nums : list, required
        List of slit number. 
    rebinned_flux : list, required
        A list containing arrays of rebinned flux. 
    median : list, required
        Calculated median (airglow). 
    
    Returns
    -----------
    all_AGSTsub_spec : list containing 1-d array of airglow and starlight subtracted spectra
        
    '''
        
    all_AGSTsub_spec = [] #contains all spectra after airglow and starlight removal 
        
    for slit in slit_nums:
        
        skysub_spectrum = rebinned_flux[slit_nums.index(slit)] - np.array(median) #airglow subtraction
        ST_con = moving_median(skysub_spectrum) #find starlight contamination
        AGSTsub_spectrum = skysub_spectrum - ST_con #remove airglow and starlight
        all_AGSTsub_spec.append(AGSTsub_spectrum)
        
    return all_AGSTsub_spec

In [ ]:
AGSTsub_spec_incl = AGST_subtraction(mask_name,slit_nums,rbflux,median) #saving 'included' sloits as FITS files
AGSTsub_spec_excl = AGST_subtraction(mask_name,slit_nums_exclude,rbflux_exclude,median) #saving 'excluded' slits as FITS files

## Saving ST and AG subtracted spectra

In [ ]:
def AGSTsub_spec_to_FITS(mask_name,AGSTsub_spec_incl,AGSTsub_spec_excl):
    
    '''
    Parameters
    ---
    mask_name : str, required
        Name of mask.
    AGSTsub_spec_incl: list, required
        INCLUDED spectra w/ airglow and starlight contamination removed.
    AGSTsub_spec_excl: list, required
        EXCLUDED spectra w/ airglow and starlight contamination removed.
    '''
    
    hdu0 = fits.PrimaryHDU()
    hdu1 = fits.ImageHDU(data=AGSTsub_spec_excl,name="RBFLUX EXCLUDE")
    hdu2 = fits.ImageHDU(data=AGSTsub_spec_incl,name="RBFLUX INCLUDE")
    hdul = fits.HDUList([hdu0,hdu1,hdu2])
    hdul.writeto("./AGST Subtracted Spectra/{0}_AGST_Subtracted_Spectra.fits.gz".format(mask_name))

In [ ]:
AGSTsub_spec_to_FITS(mask_name,AGSTsub_spec_incl,AGSTsub_spec_excl)

In [ ]:
#BREAK

In [ ]:
#BREAK

In [ ]:
#BREAK

## Find Ratios For BPT Diagram (600 ZD ONLY)

In [ ]:
def BPT_ratios(AGSTsub_spec,slit_nums): 
    
    '''
    Parameters
    ---
    AGSTsub_spec: list, req
        List containing all EXCLUDED airglow and starlight cont subtracted spectra. 
        
    Returns
    ---
    ratio_pairs: 2d-array containing pairs of ratio. Format: [NII/HAlpha,OIII/HBeta]
    '''
    
    H_Beta = np.array(AGSTsub_spec)[:,1325] #list containing all H Beta values
    OIII = np.array(AGSTsub_spec)[:,1549] #list containing all OIII values
    H_Alpha = np.array(AGSTsub_spec)[:,3943] #list containing all H Alpha values
    NII = np.array(AGSTsub_spec)[:,3975] #list containing all NII values
    
    NII_HA_ratios = NII/H_Alpha #1d array containing [NII]6584/Ha ratios
    OIII_HB_ratios = OIII/H_Beta #1d array containing [OIII]5007/Hb ratios
    
    pairing = np.vstack((NII_HA_ratios,OIII_HB_ratios)).T #create pairs:[NII/HAlpha,OIII/HBeta]
    
    ratio_pairs = []
    for element in pairing: #remove any nan (0/0) or inf (float/0). Not all slits have H_Beta or OIII
        if (any(np.isinf(element)) == True or any(np.isnan(element)) == True): 
            pass
        else:
            ratio_pairs.append(element)
    
    index_of_pairs = np.arange(len(slit_nums))[np.isin(pairing,np.array(ratio_pairs))[:,1]] #get the index of ratio pairs
    
    return np.array(ratio_pairs),index_of_pairs

In [ ]:
ratio_pairs,index_of_pairs = BPT_ratios(AGSTsub_spec_excl,slit_nums_exclude)

## Extract Redshift, QOP, and Create DF

In [ ]:
import numpy
def loadMarzResults(filepath):
    return numpy.genfromtxt(filepath, delimiter=',', skip_header=2, autostrip=True, names=True, dtype=None)

In [ ]:
#arrays containing all slits processed in MARZ
res = loadMarzResults("./Marz_Results (Redone)/{0}_Marz_KN.mz".format(mask_name)) 
confident = res[res['QOP'] == 4]

In [ ]:
redshift = [] #empty list for redshift value
QOP = [] #empty list for QOP

for n in range(len(res)): #sorting 
    redshift.append(res[n][12])
    QOP.append(res[n][13])

## Add Redshift to CSV File

In [ ]:
df = pd.DataFrame({"Mask Name":np.full(len(np.array(redshift)[index_of_pairs]),mask_name), #add mask name to DF
                   "[OIII]/H_Beta":ratio_pairs[:,1], #add [OIII]/H_Beta to DF
                   "[NII]/H_Alpha]":ratio_pairs[:,0], #add [NII]/H_Alpha] to DF
                   "Redshift":np.array(redshift)[index_of_pairs], #add redshift to DF
                   "QOP":np.array(QOP)[index_of_pairs]}) #add QOP to DF

if os.path.isfile("./BPT Diagram CSV/BPT_Ratios.csv") == False: #check if file exist. If not make it.
    df.to_csv("./BPT Diagram CSV/BPT_Ratios.csv",index=False) 
else: #if file does exist, open and add to it.
    redshift_read = pd.read_csv("./BPT Diagram CSV/BPT_Ratios.csv")
    new_csv = pd.concat([redshift_read,df])
    new_csv.to_csv("./BPT Diagram CSV/BPT_Ratios.csv",index=False)

In [ ]:
#BREAK

In [ ]:
#BREAK

In [ ]:
#BREAK

## Create BPT Diagram

In [ ]:
df_all = pd.read_csv("./BPT Diagram CSV/BPT_Ratios.csv") #read csv
QOP_3 = df_all["QOP"] == 3
OIII_HB_All = np.log10(np.array(df_all["[OIII]/H_Beta"]))[QOP_3] #contain log10 of all values of OIII/HB
NII_HA_All = np.log10(np.array(df_all["[NII]/H_Alpha]"]))[QOP_3] #contain log10 of all values of NII/HA
fig,ax = plt.subplots(1)
plt.scatter(NII_HA_All,OIII_HB_All,s=10)
ax.set_title("BPT Diagram")
ax.set_xlabel("$Log_{10}([NII]/Hα)$")
ax.set_ylabel("$Log_{10}([OIII]/Hβ)$")
#fig.set_figwidth(9)
#fig.set_figheight(9)
plt.savefig("BPT_Diagram.png")

In [ ]:
df_all = pd.read_csv("./BPT Diagram CSV/BPT_Ratios.csv") #read csv
#QOP_3 = df_all["QOP"] == 3
OIII_HB_All = np.log10(np.array(df_all["[OIII]/H_Beta"])) #contain log10 of all values of OIII/HB
NII_HA_All = np.log10(np.array(df_all["[NII]/H_Alpha]"])) #contain log10 of all values of NII/HA
fig,ax = plt.subplots(1)
plt.scatter(NII_HA_All,OIII_HB_All,s=10)
ax.set_title("BPT Diagram (QOP >= 2)")
ax.set_xlabel("$Log_{10}([NII]/Hα)$")
ax.set_ylabel("$Log_{10}([OIII]/Hβ)$")
#fig.set_figwidth(10)
#fig.set_figheight(10)
plt.savefig("BPT_Diagram (QOP>=2).png")